In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import torch
import h5py
import os
import sys
import scipy
import damselfly as df
import mayfly as mf
import scipy.signal
import scipy.stats
import scipy.interpolate

PATH = '/storage/home/adz6/group/project'
RESULTPATH = os.path.join(PATH, 'results/damselfly')
PLOTPATH = os.path.join(PATH, 'plots/damselfly')
DATAPATH = os.path.join(PATH, 'datasets/data')
#SIMDATAPATH = os.path.join(PATH, 'damselfly/data/sim_data')

"""
Date: 6/25/2021
Description: template
"""

def SaveSummedDataset(data, metadata, name):
    
    savefile = h5py.File(name, 'w')
    
    dataset = savefile.create_dataset('data', data=data)
    
    metagroup = savefile.create_group('meta')
    
    for i,key in enumerate(metadata.keys()):
        
        metagroup.create_dataset(key, data = np.array(metadata[key].array))
        
    savefile.close()
    
    

def SumDataset(MFdata, radius, gradb_freq_grid):
    
    metadata = pd.DataFrame(MFdata.metadata)
    data_shape = MFdata.data.shape
    
    summed_data = np.zeros((data_shape[0], data_shape[-1] // 60), dtype=np.complex64)
    
    summed_indexes = np.zeros(data_shape[0])
    
    pitch_angles = np.sort(metadata['theta_min'].unique())
    #print(pitch_angles)
    
    total_num_summed = 0
    
    for i, angle in enumerate(pitch_angles):
        
        inds = np.array(metadata[metadata['theta_min'] == angle].index.array)

        signal_subset = MFdata.data[inds, :]
        
        nsignal = signal_subset.shape[0]
        
        gradb_freq = InterpolateGradB(radius, angle, gradb_freq_grid)
        
        summed_signals = ShiftAndSum(signal_subset, radius, gradb_freq)
        
        summed_data[total_num_summed:total_num_summed+nsignal, :] = summed_signals
        summed_indexes[total_num_summed:total_num_summed+nsignal] = inds
        
        total_num_summed += nsignal
        
        if i % 5 == 4:
            print(f'{i+1}/{len(pitch_angles)}')
        
        
    resorted_metadata = metadata.iloc[summed_indexes]
    
    
        
    return summed_data, resorted_metadata

def InterpolateGradB(radius, pitch_angle, gradb_freq_grid):
    
    radii = gradb_freq_grid['radii']
    angles = gradb_freq_grid['angles']
    gradb_data = gradb_freq_grid['freq']
    
    interpolator = scipy.interpolate.interp2d(radii, angles, gradb_data)
    
    interpolated_freq = interpolator(radius, pitch_angle)
    
    return interpolated_freq

def ShiftAndSum(signal_subset, radius, freq):
    
    nch = 60
    signal_subset = signal_subset.reshape((signal_subset.shape[0], nch, signal_subset.shape[-1] // 60))
    nsample = signal_subset.shape[-1]
    angles = np.radians(np.arange(0, nch, 1) * 360 / nch)
    r_array = 0.10
    wavelength_lo = 3e8 / 25.86e9
    fsample = 200e6
    
    grad_b_angles = 4 * np.pi * 2 * np.pi * np.arange(0, nsample, 1) * freq / fsample
    
    x_antenna = r_array * np.cos(angles)
    y_antenna = r_array * np.sin(angles)
    
    r_electron = radius
    theta_electron = 0 + grad_b_angles
    
    x_electron = r_electron * np.cos(theta_electron)
    y_electron = r_electron * np.sin(theta_electron)
    
    #print(x_grad_b.shape, y_grad_b.shape)
    
    d_grad_b = np.sqrt((x_antenna.reshape((x_antenna.size, 1)) - x_electron.reshape((1, x_electron.size))) ** 2 + (y_antenna.reshape((y_antenna.size, 1)) - y_electron.reshape((1, y_electron.size))) ** 2)
    
    phase_shift = 2 * np.pi * (d_grad_b) / wavelength_lo + angles.reshape((angles.size, 1))
    
    shifted_signal_subset = signal_subset * np.exp(-1j * phase_shift).reshape((1, *phase_shift.shape))
    
    
    return shifted_signal_subset.sum(axis=1)
    


In [ ]:
os.listdir(os.path.join(DATAPATH, 'dl'))

# First check an old dataset created for Deep Learning

In [ ]:
old_h5file = h5py.File(os.path.join(DATAPATH, 'dl', '211015_84_1d2sl4mt.h5'))

In [ ]:
ind = 20

real = old_h5file['train']['data'][ind, 0, :]
imag = old_h5file['train']['data'][ind, 1, :]

plt.plot(real)
plt.plot(imag)
#print(h5file['train']['label'][ind])

#print(np.mean(h5file['train']['data'][140000, 2, :]))

# load data

In [ ]:
# signal data
data = mf.data.MFDataset(os.path.join(DATAPATH, '211116_grad_b_est.h5'))
metadata = pd.DataFrame(data.metadata)

# grad-b correction data
gradb_freq_grid = np.load(os.path.join(PATH, 'results/mayfly', '211129_grad_b_frequency_grid_radius_angle.npz'))

In [ ]:
data.data.shape

In [ ]:
rad = 0.001
angle = 88.0

ind = metadata[(metadata['x_min'] == rad) & (metadata['theta_min'] == angle)].index[0]

gradb_freq = InterpolateGradB(rad, angle, gradb_freq_grid)

In [ ]:
signal = data.data[ind, :]

sum_signal = ShiftAndSum(signal.reshape(1, signal.size), 0.0, gradb_freq)

In [ ]:
var = 1.38e-23 * 10 * 200e6 * 50 # variance in V squared

In [ ]:
noise = np.random.multivariate_normal([0, 0], np.eye(2) * var / 2, 8192)
noise = noise[:, 0] + 1j * noise[:, 1]

pow1 = np.mean(abs(noise) ** 2 )

noise = np.random.multivariate_normal([0, 0], np.eye(2) * var / 2, 8192 * 60)
noise = noise[:, 0] + 1j * noise[:, 1]

pow60 = np.mean(abs(noise.reshape((60, 8192)).sum(axis=0)) ** 2)
                
print(pow1, pow60, pow60 / pow1)

In [ ]:
print(np.mean(abs(np.fft.fft(noise.reshape((60, 8192)).sum(axis=0)) / 8192) ** 2), pow60, pow60 / np.mean(abs(np.fft.fft(noise.reshape((60, 8192)).sum(axis=0)) / 8192) ** 2))

In [ ]:
1.38e-23 * 10 * 200e6 * 50 / 8192

In [ ]:
plt.plot(abs(np.fft.fft(sum_signal[0, :]) / 8192) ** 2)


In [ ]:


var = 60 * 1.38e-23 * 10 * 200e6 * 50 / 8192 # var per bin

noise = np.random.multivariate_normal([0, 0], np.eye(2) * var / 2, 8192)
noise = noise[:, 0] + 1j * noise[:, 1]

plt.plot(noise.real )
plt.plot((np.fft.fftshift(np.fft.fft(sum_signal[0, :])) / 8192).real)


#plt.ylim(-0.1e-7, 0.1e-7)

In [ ]:
name = os.path.join(DATAPATH, 'bf', '211130_sens_est_dense_grid_84.5_0cm_sum.h5')
SaveSummedDataset(summed_data, summed_metadata, name)


In [ ]:
summed_metadata['theta_min'].unique()